# Transfer learning(전이학습)
이미 구축되어 있는 모델을 사용하는 기법


## 라이브러리 로드

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import random
import numpy as np
from tqdm import tqdm

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision.transforms as transforms

from multiprocessing import cpu_count
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch.nn import CrossEntropyLoss
from torchvision.models import efficientnet_b3 as efficientnet
from sklearn.model_selection import train_test_split


## 데이터 로드 및 전처리

데이터를 로드 하기 전에 기본적인 전처리 코드를 작성해줍니다

In [2]:
transform = transforms.Compose([
    #이미지 데이터를 tensor 데이터 포맷으로 바꾸어줍니다.
    transforms.ToTensor(),
    #이미지의 크기가 다를 수 있으니 크기를 통일해 줍니다.
    transforms.Resize([224,224]),
    #픽셀 단위 데이터를 정규화 시켜줍니다.
    transforms.Normalize(mean=(0.5,0.5,0.5), std = (0.5,0.5,0.5))
])

Pytorch의 ImageFoler라는 메소드를 사용하면 folder의 이름을 자동으로 라벨링 됩니다.

In [3]:
train_path = "C:/Users/User/Desktop/DataScience/data/object_image_classification_data/train/"
test_path = "C:/Users/User/Desktop/DataScience/data/object_image_classification_data/test/"
sub_path = "C:/Users/User/Desktop/DataScience/data/object_image_classification_data/sample_submission.csv"

train_data = datasets.ImageFolder(root=train_path, transform=transform)

모델 평가를 위해 train 데이터에서 vaildation 데이터를 나누어줍니다.

In [4]:
train_idx, val_idx = train_test_split(np.arange(len(train_data)), test_size = 0.2, random_state=42, shuffle=True, stratify = train_data.targets)

모델에 로드할 batch_size를 설정해줍니다.  
- batch_size : 하드웨어에 한번에 로드할 데이터의 크기
- num_workers : 데이터 로드 멀티 프로세싱을 위한 파라미터

In [5]:
#OOM(Out of Memory) 오류로 인하여 batch_size 16으로 줄임
batch_size = 16
num_workers = int(cpu_count()/2)
print("batch_size : ",batch_size)
print("num_workers : ", num_workers)

batch_size :  16
num_workers :  8


data loader를 생성해줍니다.  
- data loader : 데이터 세트를 순회하며 모델에 데이터를 넣어주는 객체

In [6]:
train_loader = DataLoader(train_data, batch_size=batch_size, sampler=SubsetRandomSampler(train_idx), num_workers=num_workers)
val_loader = DataLoader(train_data, batch_size=batch_size, sampler=SubsetRandomSampler(val_idx), num_workers=num_workers)

데이터의 크기를 확인해보겠습니다.

In [7]:
train_total = len(train_idx)
val_total = len(val_idx)

train_batches = len(train_loader)
val_batches = len(val_loader)

In [8]:
print("total train imgs", train_total, "/ total train batches", train_batches)
print("total valid imgs", val_total, "/ total val batches", val_batches)

total train imgs 40000 / total train batches 2500
total valid imgs 10000 / total val batches 625


## Device 설정

In [9]:
device = 'cuda' if torch.cuda.is_available() else print('cpu')
torch.cuda.is_available()

True

## 모델 불러오기 / 파라미터 설정

전이학습을 위한 모델을 로드합니다.  
Pytorch의 models 메소드를 사용하면 손쉽게 외부의 모델을 불러올 수 있습니다.  
Baseline에서는 efficientnet_b3모델을 사용할 것입니다.  
사전 학습 모델을 사용하는 것은 부정행위에 해당하니, pretrained=False 옵션을 설정해야 합니다.  
- pretrained=True 라면 ImageNet이라는 데이터셋을 대상으로 학습된 모델이 로드 됩니다.  


In [10]:
net = models.efficientnet_b3(pretrained=False)
net.classifier

Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)

모델에 데이터를 학습하기 위해서는 모델의 마지막 layer의 output size와 분류할 라벨의 수를 입력해주어야 합니다.  

In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

net.fc = nn.Linear(1000,10)
net =net.to(device)

모델의 파라미터를 설정해줍니다.  

In [12]:
criterion = CrossEntropyLoss()
optimizer = optim.Adam(params = net.parameters(), lr=0.0001)
epochs=10

### 학습

반복문을 이용해 학습을 진행시켜줍니다.

In [38]:
#CUDA out of memory 해결하기위하여
import gc
gc.collect()
torch.cuda.empty_cache()

for epoch in range(epochs):
    net.train()
    
    train_loss = 0
    train_correct = 0
    tqdm_dataset = tqdm(train_loader)
    for x, y in tqdm_dataset:
        x = x.to(device)
        y = y.to(device)
        outputs = net(x)
        loss = criterion(outputs, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(y).sum().item()
        
        tqdm_dataset.set_postfix({
            "Epoch" : epoch+1,
            "Loss" : '{:06f}'.format(loss.item())
        })
    
    train_loss = train_loss / train_batches
    train_acc = train_correct / train_total
    
    net.eval()
    
    val_loss= 0
    val_correct = 0
    tqdm_dataset = tqdm(val_loader)
    with torch.no_grad():
        for x,y in tqdm_dataset:
            x = x.to(device)
            y = y.to(device)
            
            outputs = net(x)
            loss = criterion(outputs, y)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(y).sum().item()
            
            tqdm_dataset.set_postfix({
                "Epoch":epoch+1,
                "Loss" : "{:06}".format(loss.item())
            })
    
    val_loss = val_loss / val_batches
    val_acc = val_correct / val_total
    
    
    print("epochs", epoch+1, "train loss", train_loss, "train acc", train_acc, "val loss", val_loss, "val acc", val_acc)

100%|██████████████████████████████████████████████| 625/625 [00:53<00:00, 11.74it/s, Epoch=1, Loss=1.1627154350280762]


epochs 1 train loss 1.8624746779680252 train acc 0.36565 val loss 1.2707061156272887 val acc 0.5457


100%|██████████████████████████████████████████████| 625/625 [00:52<00:00, 11.81it/s, Epoch=2, Loss=0.6196557283401489]


epochs 2 train loss 1.233847726893425 train acc 0.56175 val loss 0.9602255270481109 val acc 0.6629


100%|██████████████████████████████████████████████| 625/625 [00:52<00:00, 11.92it/s, Epoch=3, Loss=0.8399715423583984]


...

epochs 8 train loss 0.43066309650093315 train acc 0.850775 val loss 0.5750959768891335 val acc 0.812


100%|███████████████████████████████████████████████| 625/625 [00:52<00:00, 11.90it/s, Epoch=9, Loss=1.087569236755371]


epochs 9 train loss 0.3655903449371457 train acc 0.87405 val loss 0.6238845528900623 val acc 0.7999


100%|████████████████████████████████████████████| 625/625 [00:52<00:00, 11.97it/s, Epoch=10, Loss=0.13364143669605255]

epochs 10 train loss 0.3172193232245743 train acc 0.89035 val loss 0.6267629021435976 val acc 0.8098


## 모델 저장/ 불러오기

In [13]:
path = "C:/Users/User/Desktop/DataScience/data/object_image_classification_data/weights/model.pth"
torch.save(net.state_dict(), path)

In [14]:
net.state_dict()

OrderedDict([('features.0.0.weight',
              tensor([[[[-0.0538, -0.0189,  0.1369],
                        [-0.0057,  0.0654, -0.1517],
                        [ 0.0327,  0.0330,  0.0003]],
              
                       [[-0.0267,  0.0703,  0.0040],
                        [-0.0473,  0.1219, -0.1375],
                        [-0.0590, -0.0366, -0.0203]],
              
                       [[-0.0157,  0.0702,  0.1243],
                        [-0.0180,  0.0097,  0.0358],
                        [ 0.0352,  0.0145,  0.0865]]],
              
              
                      [[[-0.0869,  0.0852, -0.1287],
                        [-0.1474,  0.0336, -0.1454],
                        [-0.0044,  0.0401, -0.0978]],
              
                       [[ 0.1557, -0.0896, -0.0527],
                        [-0.0626,  0.0567,  0.0094],
                        [ 0.0281,  0.0073,  0.0245]],
              
                       [[ 0.0349,  0.0884,  0.0610],
                   

In [15]:
net.load_state_dict(torch.load(path))

<All keys matched successfully>

## 추론
이제 학습이 완료되었습니다.

test 데이터를 예측하겠습니다

In [16]:
from glob import glob
import PIL.Image
import numpy as np

test_images = []
for file_name in sorted(glob(test_path+"*jpg")):
    an_img = PIL.Image.open(file_name)
    img_array = np.array(an_img)
    test_images.append(img_array)

test_images = np.array(test_images)

In [30]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, transform, test_images):
        self.transform = transform
        self.img_list = test_images
        self.img_labels = [0] * 10000
        
    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        return self.transform(self.img_list[idx]), self.img_labels[idx]

In [32]:
CustomDataset(transform, test_images).


In [29]:
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers = num_workers)

test 데이터를 예측해보겠습니다.  
예측을 할 때는 학습이 진행되지 않도록 net.eval() 코드를 작성해주어야 합니다.  
데이터가 backpropagation 되어 가중치가 수정되지 않도록 해주는 코드입니다. 

In [20]:
import pandas as pd
sample_submission = pd.read_csv(sub_path)

net.eval()

batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    outputs = net(images)
    batch_index = i * batch_size
    max_vals, max_indices = torch.max(outputs, 1)
    sample_submission.iloc[batch_index:batch_index+batch_size, 1:] = max_indices.long().cpu().numpy()[:, np.newaxis]
print("예측 완료")

RuntimeError: Given groups=1, weight of size [40, 3, 3, 3], expected input[16, 32, 32, 3] to have 3 channels, but got 32 channels instead

 예측된 데이터의 라벨은 숫자로 되어있습니다.  
 train 데이터를 불러올 때 ImageFoler 메소드를 사용해 데이터를 불러왔기 때문입니다.  
 제출을 위해 라벨을 다시 복원시켜 줍니다.  

In [ ]:
labels = {
    0:'airplane',
    1:'automobile',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}

sample_submission['target']= sample_submission['target'].map(labels)

In [ ]:
dispaly(sample_submission.head())

In [ ]:
to_csv_path = "C:/Users/User/Desktop/DataScience/my_csv/object_image_classification/"
sample_submission.to_csv(to_csv_path+'baseline2_sub.csv', index=False)